# Goal

In this notebook we develop and save out the emulator for the last interglacial period. The saved emulator will be sampled (in **sample_lig_emulator.ipynb**) to constrain the parameter values that are then used to sample from the RCP8.5 scenario emulator to inform future projections of sea level rise.

The steps in this code are:

    1. Load in the LIG simulations
    2. Structure the Data for Emulation
        a. Normalize the axes
        b. Create a parameter grid mesh
    3. Define the emulator covariance and Train/condition on the data
    4. Visualize the Model Mean and Variance
    5. Save the emulator
    
We note that the structure of this emulator was chosen in **lig_cv_validation.ipynb** comparing different choices of CV functions. In this code, the structure is assumed.

# Setup

In [ ]:
# import the relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [ ]:
# define colormap
plt.set_cmap('viridis')
from lig_utilities import draw_straight_line
# define the save path for our plots
save_path='./figures/'
# set the default grid style
plt.rcParams['grid.color'] = 'k'
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.linewidth'] = 0.5

In [ ]:
# import the GPflow package for GP regression
import gpflow
from lig_utilities import normalize, denormalize, archive_gpflow_model
from datetime import datetime
# import tensorflow so we can save the trained model
import tensorflow as tf

# Load Data and Subset

In [ ]:
# define the data location
lig_path='./data/lig_data_dec18.pk1'
# load dictionary containing the data
lig_data=pickle.load( open( lig_path, "rb" ) )
# define parameter value grids
crevliq=lig_data['crevliq']
clifvmax=lig_data['clifvmax']

The LIG simulations are intended to be equilibrium runs in response to fixed forcing, and are therefore best to be compared with LIG estimates (and hence for emulation) at their final time point. We extract the data at the final time point to train the emulator:

In [ ]:
# What does this data at the final time point look like? Refer to "plot_original_sims.ipynb"
lig_data['tais'].tail(1)

In [ ]:
# get the target data at the final time point (the training target points):
ligY_train=np.asarray(lig_data['tais'].tail(1),dtype='float64')
ligY=np.asarray(ligY_train,dtype='float64').reshape(len(clifvmax),len(crevliq))

### Visualize LIG Simulated Distribution

Let's plot the histogram of the ensemble at 125ka compared with the Dutton et al. (2015/D20) bounds of [3.1-6.1 meters] at maximum AIS retreat.

In [ ]:
lig_dist_fig=plt.figure()
plt.grid()
plt.hist(np.squeeze(ligY_train),bins=np.linspace(2,7,11),normed=True)
plt.xlabel('LIG: AIS Sea Level Contributions (m)')
plt.ylabel('Density')
plt.title('Probability Distribution of AIS SL contributions during the LIG')
# plot the Dutton et al. (2015) bounds
plt.plot([3.1,3.1],[0,0.5],'r--',lw=2)
plt.plot([6.1,6.1],[0,0.5],'r--',lw=2)
plt.xlim([2.49,7.01])
plt.ylim([0,0.5])
plt.show()

### Structure the Data for Emulation

In [ ]:
# Normalize the length scales of the axes
CLIFVMAX_norm=normalize(clifvmax)
CREVLIQ_norm=normalize(crevliq)

In [ ]:
# create a meshgrid of the defined parameters and time, and then reshape
xv, yv= np.meshgrid(CLIFVMAX_norm, CREVLIQ_norm, indexing='ij')
nx,ny = len(clifvmax),len(crevliq)

# build the grid on which the data lies for emulation
X_train=np.transpose([yv,xv]).reshape(nx*ny,2)

In [ ]:
X_train[:,0].shape

# Emulation

To perform the emulation we use [GPflow](https://gpflow.readthedocs.io/en/develop/index.html), a Python package that utilizes tensorflow for computational efficiency. GPflow has the distinct advantage that out-of-the-box it permits defining components of the covariance structure along "active dimensions", specifically allowing the structure to be anisotopic if the user wants that behavior. Furthermore, GPflow assumes a prior mean function of 0, and then iterates in its optimization toward a mean structure consistent with the training data.

### Create a sample grid

We create a grid of parameter values which we can sample from to visualize our results. Note that because our training axes have been normalized, our samples are also drawn from a uniform distribution between 0 and 1, and then denormalized for plotting.

In [ ]:
# define the grid for visualization
sample_crevliq=np.linspace(-0.1,1.1,70)
sample_clifvmax=np.linspace(-0.1,1.1,70)
xx, yy = np.meshgrid(sample_clifvmax,sample_crevliq, indexing='ij')
ns1,ns2 = len(sample_clifvmax),len(sample_crevliq)
X_sample=np.transpose([yy,xx]).reshape(ns1*ns2,2)

In [ ]:
# denormalize the sample grid
denorm_x=denormalize(sample_clifvmax,np.max(clifvmax),np.min(clifvmax))
denorm_y=denormalize(sample_crevliq,np.max(crevliq),np.min(crevliq))

### Train the Emulator: k = Matern 1/2(CLIFVMAX,CREVLIQ)

In [ ]:
# create the model name
model_name='lig_model'

In [ ]:
# create an object to time the emulation
startTime = datetime.now()

# create the GP model
with gpflow.defer_build():
    
    # define a kernel and store the code for reconstruction
    k=gpflow.kernels.Matern12(2,active_dims=[0,1])
    kernel_code='gpflow.kernels.Matern12(2,active_dims=[0,1])'
    
    # create the model
    m = gpflow.models.GPR(X_train, ligY_train.reshape(np.size(ligY_train),1), kern=k, name=model_name)

# construct and compile the tensorflow session
tf.global_variables_initializer()
tf_session = m.enquire_session()
m.compile( tf_session )

#train the model
opt = gpflow.train.ScipyOptimizer()
opt.minimize(m)

# print the elapse time for the emulation
print(datetime.now() - startTime)

# archive the model (uncomment for archiving model runs)
# archive_gpflow_model(m,kernel_code,tf_session,save_dir='./archived_models/')

In [ ]:
# fix the noise term
m.likelihood.variance = 1e-06

We set the point-wise kernel variance (i.e. the nugget) to a very small number. The model log-likelihood is give by the "Objective function value" above.

Let's look at the table with the model properties and hyperparameters:

In [ ]:
# print the relevant model parameters
m.as_pandas_table()

### Visualize the Model Mean Fit

In [ ]:
# get model outputs along grid
gpr_mean,gpr_var=m.predict_y(X_sample)

# denormalize the training grid
denorm_points_x=denormalize(X_train[:,1],np.max(clifvmax),np.min(clifvmax))
denorm_points_y=denormalize(X_train[:,0],np.max(crevliq),np.min(crevliq))

In [ ]:
# choose the levels for the plot
min_val,max_val=3.0,6.5
clevels=np.linspace(min_val,max_val,11)

In [ ]:
# plot the sample
gp_mean_fig=plt.figure()
c1=plt.contourf(denorm_x,denorm_y,gpr_mean.reshape(ns1,ns2),clevels)
c2=plt.scatter(denorm_points_x,denorm_points_y,c=np.squeeze(ligY_train),s=100,norm=c1.norm,edgecolors='k')
plt.xlabel('CLIFVMAX (km/yr)')
plt.ylabel('CREVLIQ (m per (m yr$^{−1}$)$^{2}$)')
plt.title('LIG AIS Contributions to SL (m), Emulator and Simulations')
plt.ylim([-2.5,197.5])
plt.xlim([-0.2,13.2])
plt.xticks(clifvmax)
plt.yticks(crevliq)
plt.colorbar(c1)
plt.show()

In [ ]:
# save the plot out
savename='Fig2a.pdf'
gp_mean_fig.savefig(save_path+savename)

### Plot the Variance

In [ ]:
# define the levels for the plot
min_val,max_val=0,0.02
np.max(gpr_var.reshape(ns1,ns2))
clevels=np.linspace(min_val,max_val,11)

In [ ]:
np.sqrt(np.mean(gpr_var.reshape(ns1,ns2)))

In [ ]:
# plot for the square euclidean distance
gp_var_fig=plt.figure()
c1=plt.contourf(denorm_x,denorm_y,gpr_var.reshape(ns1,ns2),clevels)
plt.xlabel('CLIFVMAX (km/yr)')
plt.ylabel('CREVLIQ (m per (m yr$^{−1}$)$^{2}$)')
plt.title('LIG AIS Contributions to SL (m) Emulator and Simulations')
plt.title('LIG Emulator Variance (m$^2$)')
plt.ylim([-2.5,197.5])
plt.xlim([-0.2,13.2])
plt.xticks(clifvmax)
plt.yticks(crevliq)
plt.colorbar(c1)
plt.show()

In [ ]:
# save the plot out
savename='FigS7.pdf'
gp_var_fig.savefig(save_path+savename)

# Save

### Save the Model

The model object is a tensorflow object, so we utilize tensorflow's built in saving method.

In [ ]:
# create the saver object and archive the model object
saver = tf.train.Saver()
save_path = saver.save(tf_session, "./models/lig_model.ckpt")
print("Model saved in path: %s" % save_path)

### Save the Data to reconstruct the model

In order to reconstruct the model in **sample_lig_emulator.ipynb**, it is helpful to save out the training data which has already been conditioned in this code for emulation. This saves us multiple steps in the sampling code, and ensures we are using the identical data structure to restore the model.

In [ ]:
# store the training data
train_dat={'X_train': X_train, 'Y_train': ligY_train, 'crevliq':crevliq, 'clifvmax': clifvmax, \
          'model_name': model_name, 'kernel_code': kernel_code}

# save the training data
pickle.dump(train_dat, open( "./models/lig_model_traindata.pk1", "wb" ) )